In [ ]:
print("Hello World")
Threads.nthreads()

In [ ]:
# simple integral and derivative computing
function simpleintegral(f, a, b; dx = 0.01)
    # account for the orders of a and b
    sign = 1
    if (b-dx) < a
        sign = -1
        a, b = b, a
    end

    return sign * sum(
        [(f(x) + f(x+dx)) * dx / 2 for x in a:dx:b]
    )
end


function indefiniteintegral(f; dx = 0.01)
    # where `a` and `b` are both unbounded
    return simpleintegral(u -> f(tan(u))*sec(u), -π/2, π/2; dx=dx)

end

function simplederivative(f, x; dx = 0.01)
    return (f(x+dx) - f(x-dx)) / 2dx
end



In [ ]:
# graphing
using Plots


f(x) = sin(x)
dfdx(x) = simplederivative(f, x)
If(x) = simpleintegral(f, 0, x)

xs = 0:0.1:2π
plot(xs, f.(xs))
plot!(xs, dfdx.(xs))
plot!(xs, If.(xs) .- 1)

In [ ]:
using LinearAlgebra


# https://jakevdp.github.io/blog/2013/08/28/understanding-the-fft/

function convolute(f, g, t)
    return simpleintegral(τ -> f(t - τ) * g(τ), 0, t)
end

function indefconvolute(f, g, t; dx = 1)
    # return simpleintegral(τ -> f(t - τ) * g(τ), -10, 10)
    return indefiniteintegral(τ -> f(t - τ) * g(τ); dx=dx)
end

"""
outputs the discrete fourier transform of the signal xs,
assuming the time domain is [0:(length-1)]
and the output frequency domain is [0:(length-1)]

note that the frequency domain has the second half to be the negative frequencies
which must be shifted through the dftshift function
"""
function dft(xs)
    N = length(xs)

    ns = 0:(N-1)
    I = ns * transpose(ns)
    M = @. exp(-1im * 2pi / N * I)
    return M * xs
end


"""
dftfreq returns the frequency x-values for a dft operation
"""
function dftfreq(N; dt = 1)
    remainder = N % 2
    n = N - remainder

    # returns
    # [0, 1, ... , n/2-1,       -n/2, ... , -1] if n is even
    # [0, 1, ... , (n-1)/2, -(n-1)/2, ... , -1] if n is odd
    # length(out) = N

    posupper = n ÷ 2 - (1 - remainder)
    neglower = -n ÷ 2
    
    freqs = vcat(
        0:posupper,
        neglower:-1
    )

    return freqs ./ (dt*N)
end

"""
dftshift returns the shifted dft output array by placing the (N/2:end)
slice corrosponding to the negative frequencies at the start of the array
"""
function dftshift(xs)
    N = length(xs)

    remainder = N % 2
    n = N - remainder

    posupper = n ÷ 2 - (1 - remainder)
    neglower = -n ÷ 2

    return vcat(
        xs[(posupper+2):end],
        xs[begin:(posupper+1)]
    )
end


function fft(xs; nminsize = 32)
    # N = length(xs)
    # if !ispow2(N)
    #     throw("xs length must be a power of 2")
    # end

    # if N == 1
    #     return xs
    # else
    #     xeven = fft(xs[begin:2:end])
    #     xodd = fft(xs[2:2:end])
    #     termsrange = 0:(N÷2-1)
    #     terms = @. exp(-2im * π * termsrange / N)
    #     return vcat(
    #         xeven .+ terms .* xodd,
    #         xeven .- terms .* xodd,
    #     )
    # end

    # vectorized
    N = length(xs)

    if !ispow2(N)
        throw("xs length must be a power of 2")
    end

    Nmin = min(N, nminsize)
    ns = 0:(Nmin-1)
    I = ns * transpose(ns)
    M = @. exp(-2im * π / Nmin * I)
    # notice that each row of the right matrix contains a seperate divided array
    X = M * transpose(reshape(xs, (:, Nmin)))
    
    while size(X, 1) < N
        # the evens, odds, are lists of even/odd of each row.
        Xw = size(X, 2)
        xeven = X[:, begin:(Xw ÷ 2)]
        xodd = X[:, (Xw ÷ 2 + 1):end]

        Xh = size(X, 1)
        xrange = 0:(Xh - 1)
        factor = @. exp(-im * π / Xh * xrange)

        X = vcat(
            (xeven + xodd .* factor),
            (xeven - xodd .* factor)
        )
    end

    return collect(Iterators.flatten(X))
end


In [ ]:
using Plots

### TESTING ###

function sincosconvtest()
    # convolute sin t and cos t
    f(x) = sin(x)
    g(x) = cos(x)
    fg(t) = convolute(f, g, t)

    xs = -3π:0.1:3π
    plot(xs, fg.(xs))
end

# function doubleslittest()

# end

function sampledft()
    f(x) = cos(2π*2.3*x) + 2*cos(2π*2*x-0.5)
    # f(x) = abs(x) < 0.5 ? 1 : 0

    dt = 0.01
    xs = 0:dt:10
    N = length(xs)

    fxs = f.(xs)
    vs = dft(fxs)
    realvs = real.(vs) ./ maximum(real.(vs))
    absvs = abs.(vs) ./ maximum(abs.(vs))
    phasvs = angle.(vs)

    freqs = dftfreq(N; dt = dt)

    plot(
        xs, fxs,
        xlims = (0, 10),
        label="Source",
        title="Discrete Fourier Transform",
        alpha=0.25
    )
    xlabel!("Time/Frequency")
    ylabel!("Intensity")
    plot!(dftshift(freqs), dftshift(realvs), label="Real Freq", linewidth=2)
    plot!(dftshift(freqs), dftshift(absvs), label="Abs Freq", linewidth=3)
    plot!(dftshift(freqs), dftshift(phasvs), label="Phase Freq")
end

sampledft()

In [ ]:
# benchmarking dft
function benchmarkft()
    signal =  rand(2^10)
    @time o1 = dft(signal)
    @time o2 = fft(signal)

    # @show o1 ≈ o2
    return
end

benchmarkft()

In [ ]:
using Plots


# attempt the probability functions and operations
function makeDomain(lower, upper, step)
    return (; lower, upper, step)
end

function domainToRange(domain)
    return (domain.lower:domain.step:domain.upper)
end

function makeGaussian(mean, std)
    function gaussian(x)
        return exp(-0.5 * ((x - mean) / std) ^ 2) / (sqrt(2π) * std)
    end

    return gaussian
end

function makeBernoulli(p)
    return function (x)

        # for graphing
        # if x ≈ 0.5 || x ≈ -0.5 || x ≈ 1.5
        #     return NaN
        # end

        if round(x) == 1
            return p
        end

        if round(x) == 0
            return 1-p
        end

        return 0
    end
end

function applyntimes(f, i)
    if i == 1
        return f
    else
        return x -> applyntimes(f, i-1)(f(x))
    end
end

function makeBinominal(n, p)
    # keep adding bernoulli
    bern = makeBernoulli(p)

    for _ in 2:n
        bern = addPDFs(bern, makeBernoulli(p))
    end

    return bern
end


function plotPDF(pdf, domain)
    range = domainToRange(domain)
    return plot(range, pdf.(range), xlabel="x", ylabel="P(x)", linewidth=3)
end

function addPDFs(f, g; acc=0.1)
    return x -> indefconvolute(f, g, x; dx=acc)
end

function negatePDF(f)
    return x -> f(-x)
end

function subPDFs(f, g; acc = 0.1)
    return addPDFs(f, negatePDF(g); acc=acc)
end

function testAdd()
    domain = makeDomain(-5, 5, 0.1)
    X1 = makeGaussian(-1, 0.5)
    X2 = makeGaussian(1, 0.5)

    p1 = plotPDF(X1, domain)
    p2 = plotPDF(X2, domain)

    X3 = addPDFs(X1, X2)
    p3 = plotPDF(X3, domain)

    X4 = subPDFs(X1, X2)
    p4 = plotPDF(X4, domain)

    plot(p1, p2, p3, p4, layout = (4, 1), size = (1000, 800))
end

function testOps()
    domain = makeDomain(0, 10, 1)
    A = makeBernoulli(0.7)
    # B = makeBinominal(4, 0.7)
    B = makeBernoulli(0.7)
    B1 = addPDFs(B, makeBernoulli(0.7))
    B2 = addPDFs(B1, makeBernoulli(0.7))
    B3 = addPDFs(B2, makeBernoulli(0.7))
    B4 = addPDFs(B3, makeBernoulli(0.7))

    # p0 = plotPDF(A, domain)
    # p1 = plotPDF(B1, domain)
    # p2 = plotPDF(B2, domain)
    # p3 = plotPDF(B3, domain)
    # p4 = plotPDF(B4, domain)

    # plot(p0, p1, p2, p3, p4, layout = (5, 1), size = (1000, 2000))
    source = plotPDF(A, domain)
    p = plotPDF(makeBinominal(2, 0.7), domain)
    plot([source, p]..., layout = (2, 1))
end

testOps()